### Programming Assignment 3 
# **Logicstic Regression**

AJ Dave A. Obis and Juan Francis C. Tuatis

In [15]:
# import necessary libraries
import pandas as pd

# load the data
# define column names based on the dataset description
dataset_columns = [
    "ID", "Diagnosis", "Radius_Mean", "Texture_Mean", "Perimeter_Mean", "Area_Mean",
    "Smoothness_Mean", "Compactness_Mean", "Concavity_Mean", "Concave_Points_Mean",
    "Symmetry_Mean", "Fractal_Dimension_Mean", "Radius_SE", "Texture_SE", "Perimeter_SE",
    "Area_SE", "Smoothness_SE", "Compactness_SE", "Concavity_SE", "Concave_Points_SE",
    "Symmetry_SE", "Fractal_Dimension_SE", "Radius_Worst", "Texture_Worst", "Perimeter_Worst",
    "Area_Worst", "Smoothness_Worst", "Compactness_Worst", "Concavity_Worst", "Concave_Points_Worst",
    "Symmetry_Worst", "Fractal_Dimension_Worst"
]

# load the dataset with proper structure
data_path = "wdbc.data"
data = pd.read_csv(data_path, header=None, names=dataset_columns)

### Data Pre-Processing

In [16]:
# remove rows with missing values
data.dropna(inplace=True)

# remove the column “Sample code number”
data.drop(columns=["ID"], inplace=True)

# replace the “Class” values into 0’s and 1’s (0 for benign, 1 for malignant).
data["Diagnosis"] = data["Diagnosis"].map({"M": 1, "B": 0})

print(data.head())

   Diagnosis  Radius_Mean  Texture_Mean  Perimeter_Mean  Area_Mean  \
0          1        17.99         10.38          122.80     1001.0   
1          1        20.57         17.77          132.90     1326.0   
2          1        19.69         21.25          130.00     1203.0   
3          1        11.42         20.38           77.58      386.1   
4          1        20.29         14.34          135.10     1297.0   

   Smoothness_Mean  Compactness_Mean  Concavity_Mean  Concave_Points_Mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   Symmetry_Mean  ...  Radius_Worst  Texture_Worst  Perimeter_Worst  \
0         0.2419  ...         25.38          

# **General Guidelines**
## 1. Split the samples into 70% Training and 30% Testing at random.
